In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Vitamin_D_Levels"
cohort = "GSE33544"

# Input paths
in_trait_dir = "../../input/GEO/Vitamin_D_Levels"
in_cohort_dir = "../../input/GEO/Vitamin_D_Levels/GSE33544"

# Output paths
out_data_file = "../../output/preprocess/Vitamin_D_Levels/GSE33544.csv"
out_gene_data_file = "../../output/preprocess/Vitamin_D_Levels/gene_data/GSE33544.csv"
out_clinical_data_file = "../../output/preprocess/Vitamin_D_Levels/clinical_data/GSE33544.csv"
json_path = "../../output/preprocess/Vitamin_D_Levels/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Let's first list the directory contents to understand what files are available
import os

print("Files in the cohort directory:")
files = os.listdir(in_cohort_dir)
print(files)

# Adapt file identification to handle different naming patterns
soft_files = [f for f in files if 'soft' in f.lower() or '.soft' in f.lower() or '_soft' in f.lower()]
matrix_files = [f for f in files if 'matrix' in f.lower() or '.matrix' in f.lower() or '_matrix' in f.lower()]

# If no files with these patterns are found, look for alternative file types
if not soft_files:
    soft_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]
if not matrix_files:
    matrix_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]

print("Identified SOFT files:", soft_files)
print("Identified matrix files:", matrix_files)

# Use the first files found, if any
if len(soft_files) > 0 and len(matrix_files) > 0:
    soft_file = os.path.join(in_cohort_dir, soft_files[0])
    matrix_file = os.path.join(in_cohort_dir, matrix_files[0])
    
    # 2. Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # 3. Obtain the sample characteristics dictionary from the clinical dataframe
    sample_characteristics_dict = get_unique_values_by_row(clinical_data)
    
    # 4. Explicitly print out all the background information and the sample characteristics dictionary
    print("\nBackground Information:")
    print(background_info)
    print("\nSample Characteristics Dictionary:")
    print(sample_characteristics_dict)
else:
    print("No appropriate files found in the directory.")


Files in the cohort directory:
['GSE33544_family.soft.gz', 'GSE33544_series_matrix.txt.gz']
Identified SOFT files: ['GSE33544_family.soft.gz']
Identified matrix files: ['GSE33544_series_matrix.txt.gz']

Background Information:
!Series_title	"Human B cell receptor light chain repertoire analysis in healthy individuals and SLE patients"
!Series_summary	"Determination of expression levels of light chain V genes in peripheral blood B cells after FACS sorting for two populations of B cells (CD20+CD138-IgKappa+IgLambda- and CD20+CD138-IgKappa-IgLambda+).  Analysis was performed on healthy individuals and SLE patients with analysis performed using several models."
!Series_overall_design	"Dual channel hybridization with experimental samples detected on red channel and reference sample detected on green channel.  Two replicate hybridizations."

Sample Characteristics Dictionary:
{0: ['disease state: Healthy', 'disease state: SLE', 'disease state: N/A'], 1: ['individual: Healthy01', 'individual:

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the series title and summary, this dataset appears to contain gene expression data for
# B cell receptor light chain V genes, which makes it suitable for our analysis
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Data Availability
# For trait (Vitamin D Levels), there is no explicit measurement in the data
# The dataset focuses on B cell receptor light chain in healthy individuals and SLE patients
# It does not contain data on Vitamin D levels
trait_row = None

# For age, there is no information available in the sample characteristics
age_row = None

# For gender, there is no information available in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion
# Since the trait is not available, we'll define a placeholder conversion function
def convert_trait(val):
    # Not used because trait data is not available, but defined for completeness
    return None

def convert_age(val):
    # Not used because age data is not available, but defined for completeness
    return None

def convert_gender(val):
    # Not used because gender data is not available, but defined for completeness
    return None

# 3. Save Metadata
# Conduct initial filtering on the usability of the dataset
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# We skip this step since trait_row is None, indicating that clinical data relevant to our trait is not available


False

### Step 3: Gene Data Extraction

In [4]:
# Use the helper function to get the proper file paths
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)

# Extract gene expression data
try:
    gene_data = get_genetic_data(matrix_file_path)
    
    # Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
    
    # Print shape to understand the dataset dimensions
    print(f"\nGene expression data shape: {gene_data.shape}")
    
except Exception as e:
    print(f"Error extracting gene data: {e}")


First 20 gene/probe identifiers:
Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20'],
      dtype='object', name='ID')

Gene expression data shape: (702, 90)


### Step 4: Gene Identifier Review

In [5]:
# The identifiers in this dataset appear to be simple numeric values (1, 2, 3, etc.)
# rather than standard human gene symbols or common probe identifiers.
# These are likely to be row indices or some proprietary/custom identifiers
# that would need to be mapped to standard gene symbols.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. This part examines the data more thoroughly to determine what type of data it contains
try:
    # First, let's check a few rows of the gene_data we extracted in Step 3
    print("Sample of gene expression data (first 5 rows, first 5 columns):")
    print(gene_data.iloc[:5, :5])
    
    # Analyze the SOFT file to identify the data type and mapping information
    platform_info = []
    with gzip.open(soft_file_path, 'rt', encoding='latin-1') as f:
        for line in f:
            if line.startswith("!Platform_title") or line.startswith("!Series_title") or "description" in line.lower():
                platform_info.append(line.strip())
    
    print("\nPlatform information:")
    for line in platform_info:
        print(line)
    
    # Extract the gene annotation using the library function
    gene_annotation = get_gene_annotation(soft_file_path)
    
    # Display column names of the annotation dataframe
    print("\nGene annotation columns:")
    print(gene_annotation.columns.tolist())
    
    # Preview the annotation dataframe
    print("\nGene annotation preview:")
    annotation_preview = preview_df(gene_annotation)
    print(annotation_preview)
    
    # Check if ID column exists in the gene_annotation dataframe
    if 'ID' in gene_annotation.columns:
        # Check if any of the IDs in gene_annotation match those in gene_data
        sample_ids = list(gene_data.index[:10])
        matching_rows = gene_annotation[gene_annotation['ID'].isin(sample_ids)]
        print(f"\nMatching rows in annotation for sample IDs: {len(matching_rows)}")
        
        # Look for gene symbol column
        gene_symbol_candidates = [col for col in gene_annotation.columns if 'gene' in col.lower() or 'symbol' in col.lower() or 'name' in col.lower()]
        print(f"\nPotential gene symbol columns: {gene_symbol_candidates}")
    
except Exception as e:
    print(f"Error analyzing gene annotation data: {e}")
    gene_annotation = pd.DataFrame()

# Based on our analysis, determine if this is really gene expression data
# Check the platform description and match with the data we've extracted
is_gene_expression = False
for info in platform_info:
    if 'expression' in info.lower() or 'transcript' in info.lower() or 'mrna' in info.lower():
        is_gene_expression = True
        break

print(f"\nIs this dataset likely to contain gene expression data? {is_gene_expression}")

# If this isn't gene expression data, we need to update our metadata
if not is_gene_expression:
    print("\nNOTE: Based on our analysis, this dataset doesn't appear to contain gene expression data.")
    print("It appears to be a different type of data (possibly SNP array or other genomic data).")
    # Update is_gene_available for metadata
    is_gene_available = False
    
    # Save the updated metadata
    validate_and_save_cohort_info(
        is_final=False,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available
    )

Sample of gene expression data (first 5 rows, first 5 columns):
    GSM829558  GSM829559  GSM829560  GSM829561  GSM829562
ID                                                       
1      8.6110     7.5734     7.8586     7.0651     7.0482
2      8.8956     7.9014     7.7024     7.6270     7.2680
3      8.1202     8.5356     8.1926     7.6255     6.6475
4      7.7357     9.0515     6.9298     7.7770     6.8019
5      8.0023     9.1398     6.9036     7.9086     7.1469

Platform information:
!Series_title = Human B cell receptor light chain repertoire analysis in healthy individuals and SLE patients
!Platform_title = University of Chicago Weigert Light Chain

Gene annotation columns:
['ID', 'ORF', 'Light Chain', 'SPOT_ID', 'SEQUENCE']

Gene annotation preview:
{'ID': ['1', '2', '3', '4', '5'], 'ORF': ['A1', 'A1', 'A1', 'A1', 'A1'], 'Light Chain': ["'A1'", "'A1'", "'A1'", "'A1'", "'A1'"], 'SPOT_ID': [nan, nan, nan, nan, nan], 'SEQUENCE': ['AGGCCAATCTCCAAGGCGCCTAATTTATAAGGTTTCTAACTGGGACTCTGG